In [53]:
import random
from copy import deepcopy

In [54]:
eval('2+3*4')

14

In [55]:
class Individual:
    def __init__(self, num_operators, num_operands, lb, ub, goal):
#                     0    1    2   3  4  5  6
# 2k+1, 2k+2
#         self.code = [op1, op2, op3, a, b, c, d]
# PLUS -> +
        self.num_operators = num_operators
        self.goal = goal
        self.code = []
        for i in range(num_operators):
            self.code.append(self.random_operator())
        for i in range(num_operands):
            self.code.append(self.random_terminal(lb, ub))
        self.fitness = self.calc_fitness()
        
    def random_operator(self):
        return random.choice(['+', '-', '*'])
    
    def random_terminal(self, lb, ub):
        return random.randint(lb, ub)
    
    def __str__(self):
        return f'({self.code[3]} {self.code[1]} {self.code[4]}) {self.code[0]} ({self.code[5]} {self.code[2]} {self.code[6]})'
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def calc_fitness(self):
        value = eval(str(self))
        return -abs(value - self.goal)

In [56]:
i = Individual(3, 4, -10, 10, 137)
print(i)
print(i.fitness)

(2 - -6) - (1 * -2)
-127


In [57]:
def selection(population, tournament_size):
    participants = random.sample(population, tournament_size)
    winner = max(participants)
    return winner

In [58]:
def get_subtree(root_index, subtree_indices, num_nodes):
    if root_index < num_nodes:
        subtree_indices.append(root_index)
        get_subtree(2 * root_index + 1, subtree_indices, num_nodes)
        get_subtree(2 * root_index + 2, subtree_indices, num_nodes)

In [59]:
def crossover(parent1, parent2, child1, child2):
    root_index = random.randrange(len(parent1.code))
#     subtree_indices = [2*i+1 for i in range(root_index, )]
    subtree_indices = []
    get_subtree(root_index, subtree_indices, num_nodes=len(parent1.code))
    for i in range(len(parent1.code)):
        if i in subtree_indices:
            child1.code[i] = parent2.code[i]
            child2.code[i] = parent1.code[i]
        else:
            child1.code[i] = parent1.code[i]
            child2.code[i] = parent2.code[i]

In [71]:
def mutation(child, prob):
    for i in range(len(child.code)):
        if random.random() < prob:
            if i < child.num_operators:
                child.code[i] = child.random_operator()
            else:
                child.code[i] += child.random_terminal(-10, 10)

In [72]:
def gp(pop_size, num_iters, elitism_size, mutation_prob, goal, tournament_size):
    if (pop_size - elitism_size) % 2 == 1:
        elitism_size -= 1
    
    population = [Individual(num_operators=3, num_operands=4, lb=-10, ub=10, goal=goal) for _ in range(pop_size)]
    new_population = [Individual(num_operators=3, num_operands=4, lb=-10, ub=10, goal=goal) for _ in range(pop_size)]
    
    for it in range(num_iters):
        population.sort(reverse=True)
        if population[0].fitness == 0:
            break
        new_population[:elitism_size] = population[:elitism_size]
        
        for i in range(elitism_size, pop_size, 2):
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, new_population[i], new_population[i+1])
            
            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)
            
            new_population[i].fitness = new_population[i].calc_fitness()
            new_population[i+1].fitness = new_population[i+1].calc_fitness()
            
        population = deepcopy(new_population)
        
    best = max(population)
    print(best)
    print(best.fitness)

In [73]:
gp(50, 2000, 10, 0.1, goal=10001, tournament_size=5)

(-9 * -53) * (14 - -7)
-16
